In [76]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from keras import layers, initializers
from keras.layers import Dense,Dropout, LSTM, Flatten
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.optimizers import SGD
import keras_metrics
from sklearn import metrics
from sklearn.utils import shuffle
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from keras.layers.advanced_activations import PReLU,LeakyReLU

In [77]:
labels = pd.read_csv('labels.csv',header=0)
print(list(labels.columns.values))
print("\n")
labels= labels[pd.notnull(labels['gene'])]
# df.value_counts()

    
    
data=pd.read_csv('data.csv',header=0)
print(list(data.columns.values))
data = data[pd.notnull(data['ORF'])]

['gene', 'TCA', 'Resp', 'Ribo', 'Proteas', 'Hist', 'HTH']


['ORF', 'NAME', 'alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']


In [78]:
data.head(5)

,ORF,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,...,cold 20,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g
0,YBR166C,TYR1 TYROSINE BIOSYNTHESIS PREPHENATE DEH...,0.33,-0.17,0.04,-0.07,-0.09,-0.12,-0.03,-0.20,...,0.08,0.39,-0.17,0.23,0.20,0.20,-0.17,-0.69,0.14,-0.27
1,YOR357C,GRD19 SECRETION GOLGI PROTEIN ...,-0.64,-0.38,-0.32,-0.29,-0.22,-0.01,-0.32,-0.27,...,0.36,-0.20,-0.34,-0.14,-0.09,0.06,-0.17,0.04,-0.97,-1.79
2,YLR292C,SEC72 SECRETION ER PROTEIN TRA...,-0.23,0.19,-0.36,0.14,-0.40,0.16,-0.09,-0.12,...,0.21,-0.29,-0.36,-0.10,-0.29,-0.18,-0.34,-0.47,-0.43,-1.06
3,YGL112C,TAF60 TRANSCRIPTION TFIID 60 KD SU...,-0.69,-0.89,-0.74,-0.56,-0.64,-0.18,-0.42,-0.34,...,-0.01,-0.29,0.31,0.21,0.07,0.18,-0.14,-0.20,-0.43,-1.51
4,YIL118W,RHO3 CYTOSKELETON GTP-BINDING PR...,0.04,0.01,-0.81,NaN,-0.30,0.49,0.08,0.19,...,0.28,-0.17,0.26,0.07,-0.17,NaN,-0.10,-0.23,-0.51,-1.40


In [79]:
labels.head(5)

,gene,TCA,Resp,Ribo,Proteas,Hist,HTH
1,YAL001C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,YAL002W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,YAL003W,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,YAL005C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,YAL010C,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [80]:
X=np.loadtxt('newdata.txt')
Y=np.loadtxt('newlabels.txt')
Y=Y.astype(int)

In [81]:
print(X.shape)
print(Y.shape)
X,Y= shuffle(X,Y)

(2464, 81)
(2464,)


In [82]:
n_values = np.max(Y) + 1
newY=np.eye(n_values)[Y] #one hot encoding for softmax
print(newY.shape)

(2464, 7)


In [83]:
n=int(X.shape[0]*0.80)
length=X.shape[0]
X_train=X[0:n,:]
X_test=X[n:,:]

Y_train=Y[0:n]
Y_test=Y[n:]

newY_train=newY[0:n]
newY_test=newY[n:]

In [84]:
print(X_train.shape)
print(X_test.shape)

(1971, 81)
(493, 81)


In [85]:
print(Y_train.shape)
print(Y_test.shape)

print("One hot encoded Y shape")
print(newY_train.shape)
print(newY_test.shape)

(1971,)
(493,)
One hot encoded Y shape
(1971, 7)
(493, 7)


In [86]:
#using the sklearn class weights to find find the weights of the balanced class
Y_train=Y_train.reshape((Y_train.shape[0],))
print(Y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
Y_train=Y_train.reshape(-1, 1)

(1971,)


In [105]:
seed = 7
np.random.seed(seed)
kfold=KFold(n_splits=4, shuffle=True, random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
for train, test in kfold.split(X_train, newY_train,Y_train):
    # create model
    model = Sequential()
    model.add(Dense(175, input_dim=81, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])
    
    model.fit(X_train[train], newY_train[train], epochs=10, batch_size=10, verbose=1,class_weight=class_weights_values)
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1)
    
    labels=[0,1,2,3,4,5,6]
    #f1 values
    f1_value=metrics.f1_score(Y_train[test],predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Y_train[test],predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Y_train[test],predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Y_train[test],predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(predict_class,Y_train[test],labels,average='macro')
    recall_class_value=metrics.recall_score(predict_class,Y_train[test],labels,average=None).reshape(1, -1)
    
    scores = model.evaluate(X_train[test], newY_train[test], verbose=1)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

(1971, 81)
(1478, 81)
Epoch 1/10
1478/1478 [==============================] - 2s 1ms/step - loss: 0.5022 - acc: 0.8924
Epoch 2/10
1478/1478 [==============================] - 0s 166us/step - loss: 0.2664 - acc: 0.9344
Epoch 3/10
1478/1478 [==============================] - 0s 164us/step - loss: 0.1905 - acc: 0.9472
Epoch 4/10
1478/1478 [==============================] - 0s 165us/step - loss: 0.1758 - acc: 0.9540
Epoch 5/10
1478/1478 [==============================] - 0s 171us/step - loss: 0.1525 - acc: 0.9608
Epoch 6/10
1478/1478 [==============================] - 0s 165us/step - loss: 0.1176 - acc: 0.9648
Epoch 7/10
1478/1478 [==============================] - 0s 165us/step - loss: 0.1131 - acc: 0.9702
Epoch 8/10
1478/1478 [==============================] - 0s 165us/step - loss: 0.1153 - acc: 0.9641
Epoch 9/10
1478/1478 [==============================] - 0s 168us/step - loss: 0.1063 - acc: 0.9716
Epoch 10/10
1478/1478 [==============================] - 0s 168us/step - loss: 0.1045 - a

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


493/493 [==============================] - 1s 2ms/step
(1971, 81)
(1478, 81)
Epoch 1/10
1478/1478 [==============================] - 2s 2ms/step - loss: 0.4761 - acc: 0.8978
Epoch 2/10
1478/1478 [==============================] - 0s 162us/step - loss: 0.2345 - acc: 0.9445
Epoch 3/10
1478/1478 [==============================] - 0s 164us/step - loss: 0.1841 - acc: 0.9560
Epoch 4/10
1478/1478 [==============================] - 0s 162us/step - loss: 0.1726 - acc: 0.9581
Epoch 5/10
1478/1478 [==============================] - 0s 203us/step - loss: 0.1558 - acc: 0.9662
Epoch 6/10
1478/1478 [==============================] - 0s 162us/step - loss: 0.1376 - acc: 0.9635
Epoch 7/10
1478/1478 [==============================] - 0s 161us/step - loss: 0.1075 - acc: 0.9689
Epoch 8/10
1478/1478 [==============================] - 0s 161us/step - loss: 0.1143 - acc: 0.9668
Epoch 9/10
1478/1478 [==============================] - 0s 162us/step - loss: 0.1009 - acc: 0.9689
Epoch 10/10
1478/1478 [===========

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


493/493 [==============================] - 1s 1ms/step
(1971, 81)
(1478, 81)
Epoch 1/10
1478/1478 [==============================] - 2s 2ms/step - loss: 0.4557 - acc: 0.9019
Epoch 2/10
1478/1478 [==============================] - 0s 197us/step - loss: 0.2419 - acc: 0.9520
Epoch 3/10
1478/1478 [==============================] - 0s 166us/step - loss: 0.1893 - acc: 0.9547
Epoch 4/10
1478/1478 [==============================] - 0s 164us/step - loss: 0.1820 - acc: 0.9587
Epoch 5/10
1478/1478 [==============================] - 0s 161us/step - loss: 0.1524 - acc: 0.9608
Epoch 6/10
1478/1478 [==============================] - 0s 163us/step - loss: 0.1366 - acc: 0.9628
Epoch 7/10
1478/1478 [==============================] - 0s 164us/step - loss: 0.1111 - acc: 0.9675
Epoch 8/10
1478/1478 [==============================] - 0s 164us/step - loss: 0.1184 - acc: 0.9689
Epoch 9/10
1478/1478 [==============================] - 0s 186us/step - loss: 0.0995 - acc: 0.9743
Epoch 10/10
493/493 [=============

In [16]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9685443774413445
F1 is :0.6100011446573073
Average F1 for all the classes is:[0.98363441 0.         0.69038462 0.96682232 0.72916667 0.9
 0.        ]


In [17]:
y_pred=model.predict(X_test)
predict_class = np.argmax(y_pred, axis=1)
    
labels=[0,1,2,3,4,5,6]
#f1 values
f1_value=metrics.f1_score(Y_test,predict_class,labels,average='macro')

In [18]:
print("For test set")
print("F1 is :"+ str(np.mean(f1)))

For test set
F1 is :0.6100011446573073


In [99]:
#Based on the other models, we will compare the score against the other models
#and then see how well the model will generalise based on the test scores.

In [21]:
seed =7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #automatically balance all the classes of the training data
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(X_train[train], Y_train[train]) #sampling on the trainng data only
    X_under,y_under= shuffle(X_under,y_under)
    
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    act = PReLU(init='zero', weights=None)
    model = Sequential()
    model.add(Dense(175, input_dim=81, activation=PReLU()))
    model.add(Dropout(0.5))
    model.add(Dense(45, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.0025, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=32, verbose=1)
    
    y_pred=model.predict(X_train[test])
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(X_train[test], newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9366/9366 [==============================] - 1s 83us/step - loss: 0.4017 - acc: 0.8805 - precision: 0.8780 - recall: 0.4626
Epoch 2/10
9366/9366 [==============================] - 1s 65us/step - loss: 0.0962 - acc: 0.9732 - precision: 0.9525 - recall: 0.8700
Epoch 3/10
9366/9366 [==============================] - 1s 59us/step - loss: 0.0577 - acc: 0.9824 - precision: 0.9653 - recall: 0.9148
Epoch 4/10
9366/9366 [==============================] - 1s 68us/step - loss: 0.0421 - acc: 0.9878 - precision: 0.9768 - recall: 0.9425
Epoch 5/10
9366/9366 [==============================] - 0s 52us/step - loss: 0.0288 - acc: 0.9927 - precision: 0.9862 - recall: 0.9619
Epoch 6/10
9366/9366 [==============================] - 0s 48us/step - loss: 0.0264 - acc: 0.9931 - precision: 0.9907 - recall: 0.9604
Epoch 7/10
9366/9366 [==============================] - 0s 48us/step - loss: 0.0209 - acc: 0.9935 - precision: 0.9871 - recall: 0.9701
Epoch 8/10
9366/9366 [==============================] -

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9373/9373 [==============================] - 1s 105us/step - loss: 0.4105 - acc: 0.8701 - precision: 0.8591 - recall: 0.4600
Epoch 2/10
9373/9373 [==============================] - 1s 64us/step - loss: 0.1048 - acc: 0.9691 - precision: 0.9389 - recall: 0.8491 0s - loss: 0.1150 - acc: 0.9668 - precision: 0.9313 - re
Epoch 3/10
9373/9373 [==============================] - 0s 51us/step - loss: 0.0631 - acc: 0.9824 - precision: 0.9667 - recall: 0.9119
Epoch 4/10
9373/9373 [==============================] - 0s 53us/step - loss: 0.0381 - acc: 0.9886 - precision: 0.9784 - recall: 0.9455
Epoch 5/10
9373/9373 [==============================] - 0s 53us/step - loss: 0.0292 - acc: 0.9914 - precision: 0.9891 - recall: 0.9529
Epoch 6/10
9373/9373 [==============================] - 0s 50us/step - loss: 0.0245 - acc: 0.9925 - precision: 0.9877 - recall: 0.9612 0s - loss: 0.0246 - acc: 0.9924 - precision: 0.9860 - recall: 
Epoch 7/10
9373/9373 [==============================] - 1s 58us/step 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9373/9373 [==============================] - 1s 84us/step - loss: 0.4435 - acc: 0.8591 - precision: 0.8078 - recall: 0.4175
Epoch 2/10
9373/9373 [==============================] - 0s 49us/step - loss: 0.1253 - acc: 0.9618 - precision: 0.9177 - recall: 0.8073
Epoch 3/10
9373/9373 [==============================] - 0s 52us/step - loss: 0.0654 - acc: 0.9804 - precision: 0.9626 - recall: 0.9044
Epoch 4/10
9373/9373 [==============================] - 1s 61us/step - loss: 0.0443 - acc: 0.9868 - precision: 0.9795 - recall: 0.9261
Epoch 5/10
9373/9373 [==============================] - 1s 62us/step - loss: 0.0307 - acc: 0.9913 - precision: 0.9861 - recall: 0.9529
Epoch 6/10
9373/9373 [==============================] - 1s 57us/step - loss: 0.0264 - acc: 0.9923 - precision: 0.9840 - recall: 0.9642
Epoch 7/10
9373/9373 [==============================] - 1s 61us/step - loss: 0.0204 - acc: 0.9942 - precision: 0.9916 - recall: 0.9671
Epoch 8/10
9373/9373 [==============================] -

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9373/9373 [==============================] - 1s 89us/step - loss: 0.4376 - acc: 0.8657 - precision: 0.8247 - recall: 0.4780
Epoch 2/10
9373/9373 [==============================] - 0s 49us/step - loss: 0.1102 - acc: 0.9675 - precision: 0.9291 - recall: 0.8417
Epoch 3/10
9373/9373 [==============================] - 0s 52us/step - loss: 0.0612 - acc: 0.9799 - precision: 0.9588 - recall: 0.9037
Epoch 4/10
9373/9373 [==============================] - 0s 49us/step - loss: 0.0393 - acc: 0.9886 - precision: 0.9783 - recall: 0.9447
Epoch 5/10
9373/9373 [==============================] - 0s 48us/step - loss: 0.0340 - acc: 0.9908 - precision: 0.9816 - recall: 0.9574
Epoch 6/10
9373/9373 [==============================] - 0s 53us/step - loss: 0.0278 - acc: 0.9922 - precision: 0.9862 - recall: 0.9582
Epoch 7/10
9373/9373 [==============================] - 0s 47us/step - loss: 0.0235 - acc: 0.9940 - precision: 0.9893 - recall: 0.9694
Epoch 8/10
9373/9373 [==============================] -

In [22]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9644540860851787
F1 is :0.692161034968916
Average F1 for all the classes is:[0.98092171 0.43333333 0.702331   0.94536065 0.78318054 0.9
 0.1       ]


In [23]:
y_pred=model.predict(X_test)
predict_class = np.argmax(y_pred, axis=1)
    
labels=[0,1,2,3,4,5,6]
#f1 values
f1_value=metrics.f1_score(Y_test,predict_class,labels,average='macro')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [24]:
print("For test set")
print("F1 is :"+ str(np.mean(f1)))

For test set
F1 is :0.692161034968916


In [92]:
seed = 7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=4, shuffle=True,random_state=seed)
accuracy=[]
f1=[]
precision=[]
recall=[]

f1_class=np.empty((0,7))
precision_class=np.empty((0,7))
recall_class=np.empty((0,7))
r=2

for train, test in kfold.split(X_train, Y_train):
    # create model
    
    #y_smote=np.argmax(newY_train[train], axis=1)
    
    #pca for all features
    alphaX= X_train[train][:,0+r:18+r]
    eluX= X_train[train][:,18+r:32+r]
    edcX= X_train[train][:,32+r:47+r]
    spotX= X_train[train][:,47+r:56+r]
    spoX= X_train[train][:,56+r:58+r]
    heatX= X_train[train][:,58+r:64+r]
    dttX= X_train[train][:,64+r:68+r]
    coldX= X_train[train][:,68+r:72+r]
    diauX= X_train[train][:,72+r:79+r]

    alphaX_test= X_train[test][:,0+r:18+r]
    eluX_test= X_train[test][:,18+r:32+r]
    edcX_test= X_train[test][:,32+r:47+r]
    spotX_test= X_train[test][:,47+r:56+r]
    spoX_test= X_train[test][:,56+r:58+r]
    heatX_test= X_train[test][:,58+r:64+r]
    dttX_test= X_train[test][:,64+r:68+r]
    coldX_test= X_train[test][:,68+r:72+r]
    diauX_test= X_train[test][:,72+r:79+r]
    
    
    talphaX_test= X_test[:,0+r:18+r]
    teluX_test= X_test[:,18+r:32+r]
    tedcX_test= X_test[:,32+r:47+r]
    tspotX_test= X_test[:,47+r:56+r]
    tspoX_test= X_test[:,56+r:58+r]
    theatX_test= X_test[:,58+r:64+r]
    tdttX_test= X_test[:,64+r:68+r]
    tcoldX_test= X_test[:,68+r:72+r]
    tdiauX_test= X_test[:,72+r:79+r]
    
    pca= PCA(4)
    pcaspo = PCA(2)
    pcaed=PCA(7)
    
    alphaX= pcaed.fit_transform(alphaX)
    alphaX_test=pcaed.transform(alphaX_test)
    talphaX_test=pcaed.transform(talphaX_test)
    
    eluX= pcaed.fit_transform(eluX)
    eluX_test=pcaed.transform(eluX_test)
    teluX_test=pcaed.transform(teluX_test)
    
    edcX= pcaed.fit_transform(edcX)
    edcX_test=pcaed.transform(edcX_test)
    tedcX_test=pcaed.transform(tedcX_test)
    
    spotX= pca.fit_transform(spotX)
    spotX_test=pca.transform(spotX_test)
    tspotX_test=pca.transform(tspotX_test)
    
    spoX= pcaspo.fit_transform(spoX)
    spoX_test=pcaspo.transform(spoX_test)
    tspoX_test=pcaspo.transform(tspoX_test)
    
    heatX= pcaspo.fit_transform(heatX)
    heatX_test=pcaspo.transform(heatX_test)
    theatX_test=pcaspo.transform(theatX_test)
    
    dttX= pca.fit_transform(dttX)
    dttX_test=pca.transform(dttX_test)
    tdttX_test=pca.transform(tdttX_test)
    
    coldX= pcaspo.fit_transform(coldX)
    coldX_test=pcaspo.transform(coldX_test)
    tcoldX_test=pcaspo.transform(tcoldX_test)
    
    diauX= pca.fit_transform(diauX)
    diauX_test=pca.transform(diauX_test)
    tdiauX_test=pca.transform(tdiauX_test)
    
    name_train=X_train[train][:,0:r]
    name_test=X_train[test][:,0:r]
    final_test=X_test[:,0:r]
    
    x_train=np.concatenate([name_train,alphaX,eluX,edcX,spotX,spoX,heatX,dttX,coldX,diauX],axis=1)
    x_test=np.concatenate([name_test,alphaX_test,eluX_test,edcX_test,spotX_test,spoX_test,heatX_test,dttX_test,coldX_test,diauX_test],axis=1)
    final_test=np.concatenate([final_test,talphaX_test,teluX_test,tedcX_test,tspotX_test,tspoX_test,theatX_test,tdttX_test,tcoldX_test,tdiauX_test],axis=1)
    print(x_train.shape)
    print(x_test.shape)
    
    
    #over and undersampling-balancing all the classes
    smote_tomek = SMOTETomek(random_state=0)
    X_under, y_under=smote_tomek.fit_resample(x_train, Y_train[train])
    X_under,y_under= shuffle(X_under,y_under)
    
    n_values = np.max(y_under) + 1
    y_under=np.eye(n_values)[y_under] #one hot encoding for softmax
    
    """
    pca= PCA(50)
    X_under= pca.fit_transform(X_under)
    newX_test=pca.transform(X_train[test])
    """
    
    model = Sequential()
    model.add(Dense(175, input_dim=39+r, activation=PReLU()))
    model.add(Dropout(0.5))
    model.add(Dense(45,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    adam=optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])
    
    model.fit(X_under, y_under, epochs=10, batch_size=20, verbose=1)
    
    y_pred=model.predict(x_test)
    predict_class = np.argmax(y_pred, axis=1) #the predicted value
    
    labels=[0,1,2,3,4,5,6]
    Ylabel= np.argmax(newY_train[test], axis=1) #the actual value
    
    #f1 values
    f1_value=metrics.f1_score(Ylabel,predict_class,labels,average='macro')
    f1_class_value=metrics.f1_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #precision values
    precision_value=metrics.precision_score(Ylabel,predict_class,labels,average='macro')
    precision_class_value=metrics.precision_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
    
    #recall values
    recall_value=metrics.recall_score(Ylabel,predict_class,labels,average='macro')
    recall_class_value=metrics.recall_score(Ylabel,predict_class,labels,average=None).reshape(1, -1)
                                         
    scores = model.evaluate(x_test, newY_train[test], verbose=2)
    accuracy.append(scores[1])
    f1.append(f1_value)
    precision.append(precision_value)
    recall.append(recall_value)
    
    f1_class=np.concatenate((f1_class,f1_class_value),axis=0)
    precision_class=np.concatenate((precision_class,precision_class_value),axis=0)
    recall_class=np.concatenate((recall_class,recall_class_value),axis=0)
    
average_f1=f1_class.mean(axis=0)
average_precision=precision_class.mean(axis=0)
average_recall=recall_class.mean(axis=0)

(1476, 41)
(495, 41)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9394/9394 [==============================] - 1s 143us/step - loss: 0.4358 - acc: 0.8586 - precision: 0.8545 - recall: 0.4419
Epoch 2/10
9394/9394 [==============================] - 1s 104us/step - loss: 0.1133 - acc: 0.9669 - precision: 0.9378 - recall: 0.8428
Epoch 3/10
9394/9394 [==============================] - 1s 106us/step - loss: 0.0686 - acc: 0.9787 - precision: 0.9599 - recall: 0.8927
Epoch 4/10
9394/9394 [==============================] - 1s 93us/step - loss: 0.0501 - acc: 0.9867 - precision: 0.9790 - recall: 0.9374
Epoch 5/10
9394/9394 [==============================] - 1s 81us/step - loss: 0.0407 - acc: 0.9901 - precision: 0.9876 - recall: 0.9478
Epoch 6/10
9394/9394 [==============================] - 1s 76us/step - loss: 0.0316 - acc: 0.9919 - precision: 0.9892 - recall: 0.9583
Epoch 7/10
9394/9394 [==============================] - 1s 77us/step - loss: 0.0296 - acc: 0.9924 - precision: 0.9892 - recall: 0.9598
Epoch 8/10
9394/9394 [==============================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9394/9394 [==============================] - 1s 129us/step - loss: 0.4375 - acc: 0.8592 - precision: 0.8610 - recall: 0.4478
Epoch 2/10
9394/9394 [==============================] - 1s 82us/step - loss: 0.1087 - acc: 0.9694 - precision: 0.9486 - recall: 0.8525
Epoch 3/10
9394/9394 [==============================] - 1s 78us/step - loss: 0.0663 - acc: 0.9808 - precision: 0.9661 - recall: 0.9143
Epoch 4/10
9394/9394 [==============================] - 1s 78us/step - loss: 0.0541 - acc: 0.9849 - precision: 0.9756 - recall: 0.9225
Epoch 5/10
9394/9394 [==============================] - 1s 79us/step - loss: 0.0420 - acc: 0.9885 - precision: 0.9829 - recall: 0.9411
Epoch 6/10
9394/9394 [==============================] - 1s 78us/step - loss: 0.0333 - acc: 0.9910 - precision: 0.9869 - recall: 0.9516
Epoch 7/10
9394/9394 [==============================] - 1s 77us/step - loss: 0.0334 - acc: 0.9919 - precision: 0.9877 - recall: 0.9575
Epoch 8/10
9394/9394 [==============================] 

...

Epoch 1/10
9401/9401 [==============================] - 1s 133us/step - loss: 0.4264 - acc: 0.8633 - precision: 0.8560 - recall: 0.4646
Epoch 2/10
9401/9401 [==============================] - 1s 86us/step - loss: 0.1132 - acc: 0.9664 - precision: 0.9352 - recall: 0.8593
Epoch 3/10
9401/9401 [==============================] - 1s 85us/step - loss: 0.0703 - acc: 0.9797 - precision: 0.9671 - recall: 0.8987
Epoch 4/10
9401/9401 [==============================] - 1s 86us/step - loss: 0.0525 - acc: 0.9852 - precision: 0.9690 - recall: 0.9322
Epoch 5/10
9401/9401 [==============================] - 1s 86us/step - loss: 0.0423 - acc: 0.9879 - precision: 0.9791 - recall: 0.9404
Epoch 6/10
9401/9401 [==============================] - 1s 86us/step - loss: 0.0331 - acc: 0.9917 - precision: 0.9854 - recall: 0.9568
Epoch 7/10
9401/9401 [==============================] - 1s 85us/step - loss: 0.0315 - acc: 0.9922 - precision: 0.9892 - recall: 0.9561
Epoch 8/10
9401/9401 [==============================] 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(1480, 41)
(491, 41)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/10
9401/9401 [==============================] - 1s 132us/step - loss: 0.3968 - acc: 0.8770 - precision: 0.8366 - recall: 0.4840
Epoch 2/10
9401/9401 [==============================] - 1s 86us/step - loss: 0.1062 - acc: 0.9693 - precision: 0.9350 - recall: 0.8466
Epoch 3/10
9401/9401 [==============================] - 1s 93us/step - loss: 0.0678 - acc: 0.9807 - precision: 0.9635 - recall: 0.9047
Epoch 4/10
9401/9401 [==============================] - 1s 108us/step - loss: 0.0576 - acc: 0.9859 - precision: 0.9810 - recall: 0.9218
Epoch 5/10
9401/9401 [==============================] - 1s 94us/step - loss: 0.0433 - acc: 0.9896 - precision: 0.9867 - recall: 0.9419
Epoch 6/10
9401/9401 [==============================] - 1s 99us/step - loss: 0.0408 - acc: 0.9902 - precision: 0.9831 - recall: 0.9501
Epoch 7/10
9401/9401 [==============================] - 1s 103us/step - loss: 0.0312 - acc: 0.9906 - precision: 0.9846 - recall: 0.9523
Epoch 8/10
9401/9401 [==============================

In [100]:
print("For training:")
print("Accuracy is :"+ str(np.mean(accuracy)))
print("F1 is :"+ str(np.mean(f1)))
print("Average F1 for all the classes is:" +str(average_f1))

For training:
Accuracy is :0.9538432480121163
F1 is :0.6517129711570377
Average F1 for all the classes is:[0.9749487  0.28733766 0.71212121 0.95035256 0.77056399 0.86666667
 0.        ]


In [103]:
y_pred=model.predict(final_test)
predict_class = np.argmax(y_pred, axis=1)
labels=[0,1,2,3,4,5,6]
#f1 values
f1_value=metrics.f1_score(Y_test,predict_class,average='macro')

In [104]:
print("For test set")
print("F1 is :"+ str(np.mean(f1)))

For test set
F1 is :0.6517129711570377
